In [3]:
import pandas as pd
import json

# 예시 JSON 데이터 로딩 (리스트 형태로 여러 개의 패턴이 있다고 가정)
with open("detailed_patterns_filtered.json", "r", encoding="utf-8") as f:
    data = json.load(f)

df = pd.json_normalize(data)

In [4]:
import pandas as pd
import json
from collections import Counter
from itertools import product
import plotly.express as px
import plotly.io as pio


# 1. 언어 비율
language_counts = Counter()
for pattern in data:
    for lang in pattern.get("languages", []):
        language_counts[lang["name"]] += 1

df_lang = pd.DataFrame(language_counts.items(), columns=["Language", "Count"]).sort_values(by="Count", ascending=False).head(10)
fig1 = px.bar(df_lang, x="Language", y="Count", title="Top 10 Languages Used in Patterns")

# 2. pattern_type별 construction
pattern_construction_pairs = []
for pattern in data:
    pattern_type = pattern.get("pattern_type", "Unknown")
    constructions = pattern.get("attribute_categories", {}).get("construction", [])
    for c in constructions:
        pattern_construction_pairs.append((pattern_type, c))

df_pc = pd.DataFrame(pattern_construction_pairs, columns=["Pattern Type", "Construction"])
df_pc_count = df_pc.value_counts().reset_index(name="Count").head(20)
fig2 = px.bar(df_pc_count, x="Pattern Type", y="Count", color="Construction", barmode="group",
              title="Top 20 Pattern Type - Construction Combinations")

# 3. craft별 pattern_type
craft_type_pairs = []
for pattern in data:
    craft = pattern.get("craft", {}).get("name", "Unknown")
    pattern_type = pattern.get("pattern_type", "Unknown")
    craft_type_pairs.append((craft, pattern_type))

df_cp = pd.DataFrame(craft_type_pairs, columns=["Craft", "Pattern Type"])
df_cp_count = df_cp.value_counts().reset_index(name="Count").head(20)
fig3 = px.bar(df_cp_count, x="Craft", y="Count", color="Pattern Type", barmode="group",
              title="Top 20 Craft - Pattern Type Combinations")

fig1.show()
pio.write_image(fig1, "language_plot.png")
pio.write_image(fig2, "pattern_construction_plot.png")
fig2.show()
fig3.show()
